<a href="https://colab.research.google.com/github/andrespeedwork/Machine-Learning-Analytics/blob/main/Machine_Learning_e_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introdução

Projeto de Machine Learning: Classificação de Emails como Spam ou Não Spam

Este projeto visa desenvolver um modelo de Machine Learning para classificar automaticamente emails como spam ou não spam, ajudando a melhorar a experiência do usuário e proteger sistemas contra riscos como phishing e malware. O modelo será treinado usando um dataset de emails e explorará algoritmos como Naive Bayes, SVM e Random Forest para identificar as características que distinguem mensagens legítimas de indesejadas. O processo incluirá pré-processamento dos dados, extração de características e avaliação do desempenho com métricas como acurácia, precisão, recall e F1-score.


##Preparação do ambiente de Machine Learning & Analytics para o desenvolvimento do MVP

In [ ]:
from google.colab import drive #(Preparando o ambiente para criação do MVP Machine Learning & Analytics)
drive.mount('/content/drive'),

In [ ]:
import pandas as pd # Biblioteca fundamental para leitura, manipulação e análise de dados estruturados, especialmente em formato de tabelas (DataFrames).
import numpy as np # Biblioteca essencial para operações matemáticas e manipulação de grandes arranjos e matrizes multidimensionais, frequentemente usada para processamento de dados numéricos.
import matplotlib.pyplot as plt # Biblioteca amplamente utilizada para criação de gráficos e visualizações em 2D, essencial para análise exploratória de dados e apresentação de resultados.
from sklearn.model_selection import train_test_split # Função para dividir os dados em conjuntos de treinamento e teste, essencial para avaliar o desempenho do modelo em dados não vistos.
from sklearn.preprocessing import StandardScaler # Biblioteca responsável pela padronização dos dados, transformando as características de modo que possuam média zero e desvio padrão igual a um, melhorando a performance de modelos que dependem da escala dos dados.
from sklearn.ensemble import RandomForestClassifier # Implementação do algoritmo Random Forest (floresta aleatória), utilizado para classificação e regressão com alta robustez e capacidade de lidar com dados complexos e de alta dimensionalidade.
from sklearn.metrics import classification_report # Ferramenta para avaliar modelos de classificação, fornecendo métricas detalhadas como precisão, recall, F1-score e a matriz de confusão.
from sklearn.neighbors import KNeighborsClassifier # Implementação do algoritmo k-Nearest Neighbors (KNN), que classifica instâncias com base nas distâncias das instâncias mais próximas do conjunto de treinamento.
from sklearn.linear_model import LogisticRegression # Implementação da regressão logística, um modelo estatístico amplamente utilizado para problemas de classificação binária e multiclasse.
from sklearn.model_selection import GridSearchCV # Ferramenta para realizar a busca em grade (grid search), que ajuda a otimizar os hiperparâmetros do modelo, encontrando a melhor combinação para melhorar o desempenho do modelo.
from sklearn.model_selection import cross_val_score, StratifiedKFold # Ferramentas para realizar validação cruzada do modelo, com a opção de realizar uma divisão estratificada para garantir a distribuição proporcional das classes em cada fold.
from sklearn.feature_extraction.text import TfidfVectorizer # Ferramenta para transformar dados de texto em representações numéricas, utilizando o método TF-IDF (Term Frequency-Inverse Document Frequency), que calcula a relevância de cada termo em relação ao conjunto de documentos.
from sklearn.naive_bayes import MultinomialNB # Implementação do classificador Naive Bayes Multinomial, utilizado principalmente em problemas de classificação de texto, onde as características representam contagens ou frequências de palavras.

In [ ]:
# Ler o arquivo csv, especificando o delimitador e o tratamento de erros
df = pd.read_csv('https://raw.githubusercontent.com/andrespeedwork/Machine-Learning-Analytics/refs/heads/main/spam.csv', delimiter=',', on_bad_lines='skip')

In [ ]:
# Exibir as primeiras linhas do DataFrame
print(df.head()) # Mostra as 5 primeiras linhas por padrão e as 5 últimas

In [ ]:
# Resumo estatístico das colunas numéricas
print(df.describe())  # Estatísticas descritivas como média e desvio padrão

In [ ]:
# Verificar a dimensão do DataFrame
print(f"Dimensões do dataset: {df.shape}")  # Número de linhas e colunas

##Preparando e normalizando dados do dataset

In [ ]:
# Visualizar se há valores ausentes
print(df.isnull().sum())  # Contagem de valores nulos por coluna

In [ ]:
# Tratar valores ausentes
df = df.dropna()  # Remove linhas com valores nulos (ou use imputação)

In [ ]:
# Visualizar o dataset após o tratamento dos valores ausentes
print(df.isnull().sum())  # Contagem de valores nulos por coluna

In [ ]:
# Verificar a dimensão do DataFrame após o tratamento dos valores ausentes
print(f"Dimensões do dataset: {df.shape}")  # Número de linhas e colunas

In [ ]:
# Carregar o dataset usando delimitador
df = pd.read_csv('https://raw.githubusercontent.com/andrespeedwork/Machine-Learning-Analytics/refs/heads/main/spam.csv', encoding='latin-1', delimiter=';')
# Setando o ';' para corrigir o arquivo
df = df[['v1', 'v2']]  # Ajuste para separar rótulos e mensagens
df.columns = ['label', 'text']
df['label'] = df['label'].map({'ham': 0, 'spam': 1})  # Codificação binária

In [ ]:
# Transformando um conjunto de textos em representações numéricas
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['text'])
y = df['label']

##Dividindo o dataset entre treino e teste

In [ ]:
# Dividindo o dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Normalizar os dados (apenas X)
scaler = StandardScaler(with_mean=False) # Set with_mean=False to avoid centering
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import matplotlib.pyplot as plt
from scipy import sparse

# Assuming X is your csr_matrix
X_dense = X.toarray()  # Convert sparse matrix to a dense NumPy array

# Now you can plot the histogram
plt.hist(X_dense.flatten(), bins=20, edgecolor='black')  # Flatten for a single histogram
plt.suptitle('Distribuição das Variáveis (Features)', fontsize=16)
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming X is your csr_matrix
X_dense = X.toarray()  # Convert sparse matrix to dense NumPy array
df = pd.DataFrame(X_dense)  # Create a pandas DataFrame from the dense array

# Heatmap de correlação entre as variáveis
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()  # Calculate correlation using the DataFrame
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', cbar=True)
plt.title('Mapa de Correlação entre Variáveis', fontsize=16)
plt.show()

In [ ]:
# Modelo e validação cruzada
model = MultinomialNB()
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')

print(f"F1-Score médio: {scores.mean()}")

## Treinando e testando o modelo

Treinando o modelo com alguns algoritimos

In [ ]:
# Random Forest Classifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Fazer predições
y_pred_rf = rf.predict(X_test)

# Avaliação
print("Random Forest Classifier")
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Logistic Regression

lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# Fazer predições
y_pred_lr = lr.predict(X_test)

# Avaliação
print("Logistic Regression")
print(classification_report(y_test, y_pred_lr))

In [ ]:
# K-Nearest Neighbors (KNN)

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)

# Fazer predições
y_pred_knn = knn.predict(X_test)

# Avaliação
print("K-Nearest Neighbors")
print(classification_report(y_test, y_pred_knn))

## Ajustando Hiperparâmetros
Neste caso escolhi trabalhar com KNN

In [ ]:
# Treinar KNN com Cross-Validation

#Aqui, usamos a validação cruzada com o modelo KNN para avaliar o desempenho.

# Configurar validação cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Treinar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)  # Começando com k=5
scores = cross_val_score(knn, X, y, cv=cv, scoring='accuracy')

# Resultados
print(f"Acurácias por dobra: {scores}")
print(f"Acurácia média: {scores.mean():.4f}")

In [ ]:

# Para melhorar o desempenho, ajuste os hiperparâmetros dos modelos. Exemplos:

#    Para Random Forest: Ajuste n_estimators, max_depth, etc.
#   Para KNN: Teste diferentes valores para n_neighbors.

# Exemplo com GridSearchCV:

# from sklearn.model_selection import GridSearchCV

# Parâmetros para ajustar no KNN
param_grid = {'n_neighbors': [3, 5, 7, 9]}
grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_knn.fit(X_train, y_train)

print("Melhores parâmetros para KNN:", grid_knn.best_params_)

In [ ]:
# Ajuste do Número de Vizinhos (k)

# Teste diferentes valores de k para encontrar o melhor.

import matplotlib.pyplot as plt

k_values = range(1,15)
mean_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=cv, scoring='accuracy')
    mean_scores.append(scores.mean())

# Plotar o desempenho
plt.plot(k_values, mean_scores, marker='o')
plt.xlabel("Número de Vizinhos (k)")
plt.ylabel("Acurácia Média")
plt.title("Desempenho do KNN com diferentes valores de k")
plt.grid()
plt.show()

# Melhor k
best_k = k_values[mean_scores.index(max(mean_scores))]
print(f"Melhor valor de k: {best_k}")

In [ ]:
# Treinamento Final com o Melhor k

# Após encontrar o melhor valor de k, treine o modelo novamente e avalie o desempenho.

# Melhor KNN
knn_best = KNeighborsClassifier(n_neighbors=best_k)

# Avaliação com validação cruzada
final_scores = cross_val_score(knn_best, X, y, cv=cv, scoring='accuracy')

print(f"Melhor KNN - Acurácias por dobra: {final_scores}")
print(f"Melhor KNN - Acurácia média: {final_scores.mean():.4f}")

In [ ]:
# Avaliação no Conjunto de Teste

#Divida os dados em treino e teste para avaliar o modelo no conjunto de teste.

# Separar conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Treinar o modelo com o melhor k
knn_best.fit(X_train, y_train)
y_pred = knn_best.predict(X_test)

# Avaliar desempenho
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

**XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX** Checklist sugerido: